In [1]:
import torch
import datetime
import torch.nn as nn
from torch.nn import init
from torch.autograd import Variable
import pickle
import numpy as np
import time
import random
from collections import defaultdict
from UV_Encoders import UV_Encoder
from UV_Aggregators import UV_Aggregator
from Social_Encoders import Social_Encoder
from Social_Aggregators import Social_Aggregator
import torch.nn.functional as F
import torch.utils.data
from dataPreprocess import dataPreprocess
from model import GraphRec
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from math import sqrt
import datetime
import argparse
import os
import winsound


"""
GraphRec: Graph Neural Networks for Social Recommendation. 
Wenqi Fan, Yao Ma, Qing Li, Yuan He, Eric Zhao, Jiliang Tang, and Dawei Yin. 
In Proceedings of the 28th International Conference on World Wide Web (WWW), 2019. Preprint[https://arxiv.org/abs/1902.07243]

If you use this code, please cite our paper:
```
@inproceedings{fan2019graph,
  title={Graph Neural Networks for Social Recommendation},
  author={Fan, Wenqi and Ma, Yao and Li, Qing and He, Yuan and Zhao, Eric and Tang, Jiliang and Yin, Dawei},
  booktitle={WWW},
  year={2019}
}
```

"""
          #graph         
def train(model, device, train_loader, optimizer, epoch, best_rmse, best_mae):
    model.train()
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        batch_nodes_u, batch_nodes_v, labels_list = data
        optimizer.zero_grad()
        loss = model.loss(batch_nodes_u.to(device), batch_nodes_v.to(device), labels_list.to(device))
        loss.backward(retain_graph=True)
        optimizer.step()
        running_loss += loss.item()
        if i % 100 == 0:
            print('[%d, %5d] loss: %.3f, The best rmse/mae: %.6f / %.6f' % (
                epoch, i, running_loss / 100, best_rmse, best_mae))
            running_loss = 0.0
    return 0


def test(model, device, test_loader):
    model.eval()
    tmp_pred = []
    target = []
    with torch.no_grad():
        for test_u, test_v, tmp_target in test_loader:
            test_u, test_v, tmp_target = test_u.to(device), test_v.to(device), tmp_target.to(device)
            val_output = model.forward(test_u, test_v)
            tmp_pred.append(list(val_output.data.cpu().numpy()))
            target.append(list(tmp_target.data.cpu().numpy()))
    tmp_pred = np.array(sum(tmp_pred, []))
    target = np.array(sum(target, []))
    expected_rmse = sqrt(mean_squared_error(tmp_pred, target))
    mae = mean_absolute_error(tmp_pred, target)
    print('tmp_pred\n',tmp_pred)
    print('target\n',target)
    
    return expected_rmse, mae,tmp_pred,target

In [2]:
print('begin')
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
use_cuda = False
if torch.cuda.is_available():
    use_cuda = True
    print('True')
device = torch.device("cuda" if use_cuda else "cpu")

print('device',device)

begin
True
device cuda


In [3]:
batch_size=128
embed_dim=64
lr=0.001
test_batch_size=1000
epochs=100
data='data/Toy'
embed_dim = embed_dim
dir_data = data
path_data = ''


In [4]:

for file in os.listdir(dir_data):
    if file.endswith(".pickle"):
        path_data = dir_data + '/' + file
        print("\nPickle File Exists, No need to generate one....\n")
if path_data == '':
    print('\ndataset.pickle not found. Generating a pickle file.......\n')
    dp = dataPreprocess(dir_data)
    print('a')
    dp.preprocess()
    print('b')
    path_data = dir_data + '/dataset.pickle'
print('\n***** Data Loaded ****\n')
       


Pickle File Exists, No need to generate one....


***** Data Loaded ****



In [5]:
 
data_file = open(path_data, 'rb')
if not dir_data == 'data/Toy':
    print(1)
    history_u_lists = pickle.load(data_file)
    history_ur_lists = pickle.load(data_file)
    history_v_lists = pickle.load(data_file)
    history_vr_lists = pickle.load(data_file)
    train_u = pickle.load(data_file)
    train_v = pickle.load(data_file)
    train_r = pickle.load(data_file)
    test_u = pickle.load(data_file)
    test_v = pickle.load(data_file)
    test_r = pickle.load(data_file)
    social_adj_lists = pickle.load(data_file)
    ratings_list = pickle.load(data_file)
    users = pickle.load(data_file)
    friends = pickle.load(data_file)
    trust = pickle.load(data_file)
    usersL = list(set(users+friends))
    num_users = max(max(usersL), max(train_u), max(test_u))
    num_items = max(max(list(train_v)), max(list(test_v)))
    num_ratings = ratings_list.__len__()
    
else:
    print(2)
    history_u_lists, history_ur_lists, history_v_lists, history_vr_lists, train_u,train_v, train_r, test_u, test_v, test_r, social_adj_lists, ratings_list = pickle.load(data_file)
    num_users = history_u_lists.__len__()
    num_items = history_v_lists.__len__()
    num_ratings = ratings_list.__len__()

2


In [6]:
trainset = torch.utils.data.TensorDataset(torch.LongTensor(train_u), torch.LongTensor(train_v),
                                            torch.FloatTensor(train_r))

testset = torch.utils.data.TensorDataset(torch.LongTensor(test_u), torch.LongTensor(test_v),
                                            torch.FloatTensor(test_r))

train_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(testset, batch_size=test_batch_size, shuffle=True)


In [7]:

u2e = nn.Embedding(num_users, embed_dim).to(device)
v2e = nn.Embedding(num_items, embed_dim).to(device)
r2e = nn.Embedding(num_ratings, embed_dim).to(device)


first_embeddings=u2e,v2e,r2e

In [44]:
print(u2e)
print(v2e)
print(r2e)

print(len(history_u_lists))
print(len(history_v_lists))
print(len(ratings_list))

print(len(train_u))
print(len(train_v))
print(len(train_r))

Embedding(705, 64)
Embedding(1941, 64)
Embedding(8, 64)
705
1941
8
14091
14091
14091


In [9]:
agg_u_history = UV_Aggregator(v2e, r2e, u2e, embed_dim, cuda=device, uv=True)
enc_u_history = UV_Encoder(u2e, embed_dim, history_u_lists, history_ur_lists, agg_u_history, cuda=device, uv=True)

agg_u_social = Social_Aggregator(lambda nodes: enc_u_history(nodes).t(), u2e, embed_dim, cuda=device)
enc_u = Social_Encoder(lambda nodes: enc_u_history(nodes).t(), embed_dim, social_adj_lists, agg_u_social,
                        base_model=enc_u_history, cuda=device)
agg_v_history = UV_Aggregator(v2e, r2e, u2e, embed_dim, cuda=device, uv=False)
enc_v_history = UV_Encoder(v2e, embed_dim, history_v_lists, history_vr_lists, agg_v_history, cuda=device, uv=False)


In [10]:
print(u2e.weight[1][1])
print(v2e.weight[1][1])
print(r2e.weight[1][1])

tensor(0.6939, device='cuda:0', grad_fn=<SelectBackward0>)
tensor(0.6848, device='cuda:0', grad_fn=<SelectBackward0>)
tensor(0.1529, device='cuda:0', grad_fn=<SelectBackward0>)


In [11]:


# model
graphrec = GraphRec(enc_u, enc_v_history, r2e).to(device)
optimizer = torch.optim.RMSprop(graphrec.parameters(), lr=lr, alpha=0.9)




best_embeddings=u2e,v2e,r2e


best_rmse = 9999.0
best_mae = 9999.0
endure_count = 0
RR=range(1, epochs + 1)
Length=len(RR)
epoch=1

In [ ]:

for epoch in RR:
    print('epoch :',epoch,'/',Length,' : ',datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
    print('train')
    train(graphrec, device, train_loader, optimizer, epoch, best_rmse, best_mae)
    

    
    print('test')
    expected_rmse, mae ,tmp_pred,target= test(graphrec, device, test_loader)
    # please add the validation set to tune the hyper-parameters based on your datasets.
    # early stopping (no validation set in toy dataset)
    if best_rmse > expected_rmse:
        best_rmse = expected_rmse
        best_mae = mae
        endure_count = 0
        best_embeddings=u2e,v2e,r2e
    else:
        endure_count += 1

    print("rmse: %.4f, mae:%.4f " % (expected_rmse, mae))

    
    winsound.Beep(440,1000)
    
    if endure_count > 5:
        filehandler = open('best_embeddings(u2e-v2e-r2e)', 'w') 
        filehandler = open('', 'w') 
        pickle.dump(best_embeddings, filehandler)
        filehandler.close()
        break



In [29]:
history_u_lists[1],history_ur_lists[1]

([34, 60, 102, 0, 52, 46, 33, 45, 145, 6, 68],
 [3, 2, 4, 4, 2, 3, 3, 4, 4, 5, 2])

In [77]:
type(history_u_lists)

collections.defaultdict

In [146]:
usersRating=[]
usersRatingEncoded=[]

In [147]:

for i in range(len(history_u_lists)):
    usersRating.append([])
    usersRatingEncoded.append([])
    for j in range(len(history_u_lists[i])):
        usersRating[i].append((history_u_lists[i][j],history_ur_lists[i][j]))
        usersRatingEncoded[i].append((history_u_lists[i][j]*8+history_ur_lists[i][j]))
        print([i,history_u_lists[i][j],history_ur_lists[i][j]])
        print(history_u_lists[i][j]*8+history_ur_lists[i][j])
    usersRatingEncoded[i].sort()
        


[0, 55, 0]
440
[1, 34, 3]
275
[1, 60, 2]
482
[1, 102, 4]
820
[1, 0, 4]
4
[1, 52, 2]
418
[1, 46, 3]
371
[1, 33, 3]
267
[1, 45, 4]
364
[1, 145, 4]
1164
[1, 6, 5]
53
[1, 68, 2]
546
[2, 60, 4]
484
[2, 34, 3]
275
[2, 11, 0]
88
[2, 52, 5]
421
[2, 76, 4]
612
[2, 62, 5]
501
[2, 102, 0]
816
[2, 0, 0]
0
[2, 45, 3]
363
[3, 203, 3]
1627
[3, 52, 5]
421
[3, 33, 2]
266
[3, 24, 3]
195
[4, 31, 0]
248
[4, 0, 4]
4
[4, 33, 5]
269
[4, 88, 4]
708
[4, 51, 2]
410
[5, 74, 2]
594
[5, 64, 6]
518
[5, 69, 5]
557
[5, 62, 3]
499
[5, 47, 3]
379
[5, 0, 3]
3
[6, 55, 0]
440
[6, 26, 1]
209
[6, 34, 3]
275
[6, 31, 3]
251
[6, 24, 4]
196
[6, 144, 2]
1154
[6, 62, 3]
499
[6, 47, 3]
379
[6, 68, 4]
548
[6, 45, 0]
360
[7, 24, 3]
195
[8, 74, 5]
597
[8, 62, 3]
499
[8, 31, 2]
250
[8, 86, 4]
692
[8, 26, 5]
213
[8, 45, 2]
362
[8, 46, 5]
373
[8, 32, 2]
258
[8, 102, 0]
816
[8, 55, 5]
445
[8, 69, 5]
557
[8, 10, 3]
83
[8, 52, 4]
420
[8, 76, 3]
611
[9, 40, 5]
325
[9, 689, 3]
5515
[9, 52, 3]
419
[9, 1132, 5]
9061
[9, 1140, 3]
9123
[9, 11, 3

In [148]:
print(usersRating[704],usersRating[3])
print(usersRatingEncoded[704],usersRatingEncoded[3])


[(102, 3), (55, 3), (80, 3), (74, 3), (62, 3), (31, 3)] [(203, 3), (52, 5), (33, 2), (24, 3)]
[251, 443, 499, 595, 643, 819] [195, 266, 421, 1627]


In [157]:
itemsRating=[]

In [158]:

for i in range(len(history_v_lists)):
    itemsRating.append([])
    for j in range(len(history_v_lists[i])):
        itemsRating[i].append((history_v_lists[i][j],history_vr_lists[i][j]))
        print([i,history_v_lists[i][j],history_vr_lists[i][j]])


[0, 681, 0]
[0, 48, 2]
[0, 577, 4]
[0, 143, 6]
[0, 615, 0]
[0, 374, 5]
[0, 285, 2]
[0, 19, 3]
[0, 379, 3]
[0, 369, 2]
[0, 55, 2]
[0, 638, 2]
[0, 413, 7]
[0, 213, 1]
[0, 146, 5]
[0, 375, 4]
[0, 159, 6]
[0, 346, 4]
[0, 66, 0]
[0, 587, 5]
[0, 620, 1]
[0, 203, 1]
[0, 113, 3]
[0, 15, 5]
[0, 234, 5]
[0, 636, 4]
[0, 359, 4]
[0, 76, 4]
[0, 486, 2]
[0, 78, 4]
[0, 320, 3]
[0, 70, 2]
[0, 598, 1]
[0, 321, 3]
[0, 650, 0]
[0, 676, 2]
[0, 226, 6]
[0, 105, 4]
[0, 641, 6]
[0, 252, 3]
[0, 350, 5]
[0, 131, 0]
[0, 611, 3]
[0, 454, 5]
[0, 590, 0]
[0, 570, 4]
[0, 703, 3]
[0, 405, 0]
[0, 227, 6]
[0, 11, 2]
[0, 503, 3]
[0, 274, 7]
[0, 429, 5]
[0, 694, 3]
[0, 126, 6]
[0, 255, 3]
[0, 680, 4]
[0, 52, 6]
[0, 51, 0]
[0, 258, 2]
[0, 300, 7]
[0, 151, 4]
[0, 1, 4]
[0, 699, 2]
[0, 568, 0]
[0, 585, 2]
[0, 516, 3]
[0, 632, 4]
[0, 551, 6]
[0, 425, 2]
[0, 659, 4]
[0, 343, 6]
[0, 325, 5]
[0, 646, 4]
[0, 393, 2]
[0, 479, 3]
[0, 200, 0]
[0, 138, 4]
[0, 404, 4]
[0, 483, 5]
[0, 286, 6]
[0, 164, 0]
[0, 336, 2]
[0, 648, 0]
[0, 2

In [159]:
itemsRating[2]

[(172, 2), (193, 7), (37, 2), (53, 4), (292, 4)]

In [160]:
print(290,social_adj_lists[290])
print(497,social_adj_lists[497])


290 {289, 497}
497 {290, 388}


In [161]:
usersRating[290],usersRating[497]

([(11, 3), (8, 5), (47, 3), (45, 3), (0, 5), (13, 2)],
 [(57, 5), (76, 2), (31, 3), (24, 5), (62, 2)])

In [162]:
# item * 8 + id_de_rating
0*8+0,0*8+7,1*8+0,1*8+7

(0, 7, 8, 15)

In [208]:
def addTrustBetween(social_adj_lists, i,j):
    social_adj_lists[j].add(i)
    social_adj_lists[i].add(j)

trustDegree={}
def addTrustDegreeBetween(i,j,trustDegree):
    if i>j:
        k=i
        i=j
        j=k
    if str((i,j)) not in trustDegree:
        print('not')
        trustDegree[str((i,j))]=1
    else:
        print('in')
        trustDegree[str((i,j))]=trustDegree[str((i,j))]+1
    



In [216]:
for user,userRating in enumerate(usersRating):
    # print(user)
    for i,couple_item_rating in enumerate(userRating):
        # print(i,couple_item_rating)
        
        item=couple_item_rating[0]
        rating=couple_item_rating[1]
        #print('\t',itemsRating[item])
        for user_, rating_ in itemsRating[item]:
            # print(user,rating)
            if user!=user_ and abs(rating-rating_) < 2:
                print(user,rating)
                addTrustDegreeBetween(user,user_,trustDegree)

    


In [210]:
trustDegree

{}

In [174]:
itemRating=itemsRating[1]

In [175]:
itemRating

[(81, 1), (274, 7), (184, 4), (619, 7)]

In [179]:
for user, rating in itemRating:
    print(user,rating)
    

81 1
274 7
184 4
619 7


In [195]:
l={}
l[str((1,23))]


KeyError: '(1, 23)'

In [196]:
if str((1,2)) not in l:
    print('not')
else:
    print('in')
    

not


In [198]:
l[str((1,2))]=1


In [203]:
l[str((1,2))]=l[str((1,2))]+1
l[str((1,2))]


6

In [204]:
type(l)

dict